In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import copy
import importlib
import get_lens1_rcp85 as lens1
import sst_biases_2018pub as sstbias
importlib.reload( sstbias )
importlib.reload( lens1 )

In [ ]:
monthly_mean_bias,lat_sst,lon_sst = sstbias.original()

In [ ]:
ts,landf,lat,lon = lens1.FullTS()
#ts=ts[0:30,:,:,:]
tsC=copy.deepcopy( ts )

In [ ]:
tsPD,landfPD,latPD,lonPD = lens1.PresentDayTS()
#ts=ts[0:30,:,:,:]
tsPDC=copy.deepcopy( tsPD )

In [ ]:
pub_ens_x=[0,8,11,21,24,29,-1]
SSTlab = ['SST1' , 'SST2' , 'SST3' , 'SST4' , 'SST5' , 'SST6' , 'SST7' ]

In [ ]:
nens,nt,ny,nx = np.shape( ts )
nYr =  nt//12 
print(nens,nYr,ny,nx)

tsC = tsC.reshape( nens ,nYr, 12, ny, nx )
tsR = np.zeros( (nens ,nYr, 12, ny, nx ) )

for nen in np.arange( nens ):
    for iy in np.arange( nYr ):
        for im in np.arange( 12 ):
            tsR[nen,iy,im,:,:] = tsC[nen,iy,im,:,:] - monthly_mean_bias[im,:,:]

In [ ]:
nensPD,ntPD,ny,nx = np.shape( tsPD )
nYrPD =  ntPD//12 
print(nensPD,nYrPD,ny,nx)

tsPDC = tsPDC.reshape( nensPD ,nYrPD, 12, ny, nx )
tsPDR = np.zeros( (nensPD ,nYrPD, 12, ny, nx ) )

for nen in np.arange( nensPD ):
    for iy in np.arange( nYrPD ):
        for im in np.arange( 12 ):
            tsPDR[nen,iy,im,:,:] = tsPDC[nen,iy,im,:,:] - monthly_mean_bias[im,:,:]

## AT this point you've read in the LENS TS data for 2070-2099 with pre-pended TS from an ealrier CESM1 coupled run - in ts and tsC
## Also have present day LENS in tsPD and tsPDC 
## And aslo have bias corrected TS's in tsR and tsPDR

In [ ]:
# ooooohhhh 

# 'Center' the data

In [ ]:
tsCpub=tsC[pub_ens_x,:,:,:]

In [ ]:
print(np.shape(tsCpub))
nens,nYr,nmo,ny,nx = np.shape(tsCpub)


In [ ]:
ts_1=tsCpub[0,:,:,:,:].reshape((nYr*nmo,ny,nx))
#ts_1a=ts_1.mean( axis=1 )
print( )

In [ ]:
# Don't be bothered by apparent dimension mismatch.
# This line 'broadcasts' the mean taken for axis=0
# to the other dimensions, i.e., this line is equivalent 
# to subtracting 2 3D arrays where the mean array is 
# a stack of identical 2D arrays. Thanks ChatGPT.
#-----------------------------------------------------
ts_1x = ts_1 - ts_1.mean(axis=0)


In [ ]:
nt,ny,nx=np.shape(ts_1)
for t in np.arange( nt ):
    ts_1x[t, :,:] = np.where( landf < 0.05 , ts_1x[t,:,:], 0. )

In [ ]:
# Scale with SQRT(cos(lat))
Lon,Lat=np.meshgrid(lon,lat) 
CosScale=np.sqrt( np.cos( (np.pi/180.)*Lat ) )
print(np.shape(CosScale))
plt.contour( CosScale )

In [ ]:
# More "broadcasting"
ts_1xC = ts_1x * CosScale

In [ ]:
#plt.plot( tsoo[:,90,90]-304.)
plt.contour( ts_1xC[0,:,:],levels=31)
plt.colorbar()


In [ ]:
ts_1xC=np.reshape( ts_1xC , (nt, ny*nx) )

In [ ]:
u,s,vt=np.linalg.svd( ts_1xC, full_matrices=False )

In [ ]:
print(np.shape(u))
print(np.shape(s))
print(np.shape(vt))
#plt.plot(np.average(u,axis=1))
#plt.plot( u.flatten() )

In [ ]:
#print( np.shape(vt))
fig=plt.figure(figsize=(20,10))

lv=31 #np.linspace( -.03,.03,num=31)

for n in np.arange(12):
    ax1 = fig.add_subplot(3, 4, n+1 ) # , projection=projPC )
    co=ax1.contourf( vt[n,:].reshape(ny,nx) , levels=lv)
    plt.colorbar(co ) #,cax=cax,orientation='horizontal')


In [ ]:
## 
#plt.plot(u[0,:])
plt.plot(u[:,6])

In [ ]:
from scipy.signal import convolve


In [ ]:
wsiz=12
kernel = np.ones((wsiz,1)) / wsiz
smoop = convolve(u, kernel, mode='same')

In [ ]:
i=8
plt.plot( u[:,i])
plt.plot( smoop[:,i])

In [ ]:
from scipy.signal import detrend

# Assume `data` is your 3D (time, lat, lon) numpy array
# data.shape might be something like (1000, 180, 360) for 1000 time steps, 180 latitude steps, and 360 longitude steps

# Initialize an array to hold detrended data
detrended_data = np.empty_like(data)

# Iterate over lat and lon
for i in range(data.shape[1]):
    for j in range(data.shape[2]):
        detrended_data[:, i, j] = detrend(data[:, i, j])



In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
pca=PCA(n_components=20)

In [ ]:
tsoov=pca.fit_transform(tsoop)

In [ ]:
print(np.shape(pca.components_))

In [ ]:
plt.plot( pca.components_[0,:])
plt.plot( vt[0,:])


In [ ]:
print(192*288)

In [ ]:
skoov=np.reshape( pca.components_ , (20,192,288) )

In [ ]:
plt.contour(skoov[14,:,:])